<a href="https://colab.research.google.com/github/gabrielcz6/supermarketsalesanalysis/blob/main/supermarketanalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello")



print("edicion2prueba")

hello
